In [167]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [168]:
LOCAL_GECKO_PATH = r'/Users/meithnv/Library/Caches/Homebrew/downloads/geckodriver/0.31.0/bin/geckodriver'


In [169]:
def loadInfoWithDelay(url, waitForItem, gecko_path):
    driver = webdriver.Firefox(executable_path=gecko_path)
    driver.get(url)
    timeout = 200

    try:
        element_present = EC.presence_of_element_located((By.CLASS_NAME, waitForItem))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print( "Timed out waiting for page to load")

    page = driver.page_source
    soup = BeautifulSoup(page, "lxml")

    # CLOSE PAGE
    
    return soup, driver


In [170]:
# SINGLE PAPER DETAILS

def fetchPaperDetails(url, paperId):
    paper_det ={}
    final_pg, driver = loadInfoWithDelay(url, 'note_content_value', LOCAL_GECKO_PATH)

    header = final_pg.find('h2', attrs={'class':'note_content_title'}).find('a').string
    pdf_link = final_pg.find('a', attrs={'class':'note_content_pdf'}).get('href')
    abstract = final_pg.find('span', attrs={'class':'note_content_value'}).string
    reviews = final_pg.findAll('div', attrs={'class':'comment-level-odd'})

    paper_reviews = []
    for r in reviews:
        # print(r)
        review ={}
        review["reviewer"] = r.find('span', attrs={'class':'signatures'}).string
        node_contents = r.findAll('div', attrs={'class':'note_contents'})
        # print(node_contents)
        for attr in node_contents:
            # print(attr)
            field = attr.find('span', attrs={'class':'note_content_field'})
            val = attr.find('span', attrs={'class':'note_content_value'})
            for f, v in zip(field ,val):
                review[str(f).split(":")[0].strip()] = v

        # print(review)
        # print('\n\n--')
        paper_reviews.append(review)
    
    # SAVE PAPER DETAILS
    paper_det['paper_id'] = paperId   
    paper_det['header'] = header 
    paper_det['abstract'] = abstract 
    paper_det['reviews'] = paper_reviews
    paper_det['url'] = pdf_link
    driver.quit()

    return paper_det

In [171]:
def savePaperJSON(obj , year, venue_type, tab_type):
    
    file_path = str(year)+'/'+venue_type
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    
    store_path = file_path+'/'+obj['paper_id']+'.json'
    
    with open(store_path, 'w') as fp:
        json.dump(obj, fp)
        fp.close()


In [177]:
# 2016
BASE_URL = 'https://openreview.net'
start_url = BASE_URL+'/group?id=ICLR.cc/2016/workshop'
soup, driver = loadInfoWithDelay(start_url, 'pagination', LOCAL_GECKO_PATH)


/var/folders/2v/8202bzhj5lbd5dyqgq4rzt7r0000gn/T/ipykernel_27977/1499548198.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=gecko_path)


In [178]:
pages = soup.findAll('ul', attrs={'class':'pagination'})[0].findAll('li')
pages = len(pages[2:len(pages)-2])
# pages

for page in range(pages-1):

    timeout = 200
    print(page)
    try:
        

        element_present = EC.presence_of_element_located((By.CLASS_NAME, 'note'))
        WebDriverWait(driver, timeout).until((element_present))

        element_present = EC.presence_of_element_located((By.CSS_SELECTOR, 'li.right-arrow a'))
        WebDriverWait(driver, timeout).until((element_present))
        # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.right-arrow a'))).click()
        page = driver.page_source

        soup = BeautifulSoup(page, "lxml")
        all_papers =soup.findAll('h4')
        print(all_papers)

        # for paper in all_papers[1:]:
        #         print(" >>> PAPER : "+paper.find('a').string.strip() )
        #         paper_link = BASE_URL+paper.find('a').get('href')
        #         paperID = paper.find('a').get('href').split("=")[-1]
        #         print(paper_link)

        #         # FINAL PAPER PAGE 
        #         paper_details = fetchPaperDetails(paper_link, paperID)
        #         print(paper_details)

        #         # STORE THIS IN FILE
        #         savePaperJSON(paper_details , '2016', 'workshop', '')
        
       
       
        # NEXT PAGE 
        page_btn = driver.find_element(By.CSS_SELECTOR, 'li.right-arrow a')
        # print(page_btn)
        page_btn.click()
        
      
    except TimeoutException:
        print( "Timed out waiting for page to load")
   

# driver.quit()
# tabs = soup.findAll('h4')[1:]
# tabs
# for tab in tabs: 
#     print("  >> "+tab.get('href'))
#     tab_type = str(tab.get('href')).split("#")[-1]
#     link_tab = conference_link+tab.get('href')
#     all_papers_pg = loadInfoWithDelay(link_tab, 'note', LOCAL_GECKO_PATH)

0
[<h4>
<span class="venue-location">
<span aria-hidden="true" class="glyphicon glyphicon-globe"></span> Caribe Hilton, San Juan, Puerto Rico
      </span>
<span class="venue-date">
<span aria-hidden="true" class="glyphicon glyphicon-calendar"></span> May 2 - 4, 2016
      </span>
<span class="venue-website">
<span aria-hidden="true" class="glyphicon glyphicon-new-window"></span> <a href="http://www.iclr.cc" target="_blank" title="ICLR 2016 - Workshop Track Homepage">http://www.iclr.cc</a>
</span>
</h4>, <h4>
<a href="/forum?id=3QxgDBPQxIp7y9wltPq9">
              Fully Convolutional Nerual Network for Body Part Segmentation
          </a>
<a class="pdf-link" href="/pdf?id=3QxgDBPQxIp7y9wltPq9" target="_blank" title="Download PDF"><img src="/images/pdf_icon_blue.svg"/></a>
</h4>, <h4>
<a href="/forum?id=oVgoWgzpZSrlgPMRsB19">
              A metric learning approach for graph-based label propagation
          </a>
<a class="pdf-link" href="/pdf?id=oVgoWgzpZSrlgPMRsB19" target="_blank" 

ElementClickInterceptedException: Message: Element <a href="#"> is not clickable at point (713,835) because another element <div class="content-overlay"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.jsm:12:1
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:192:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.jsm:291:5
webdriverClickElement@chrome://remote/content/marionette/interaction.js:166:11
interaction.clickElement@chrome://remote/content/marionette/interaction.js:125:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:92:31
